In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from pyscipopt import Model
import time

In [2]:
class box:
    def __init__(self,dim):
        self.dimensioni=dim

    def getW(self):
        return self.dimensioni[0]

    def getD(self):
        return self.dimensioni[1]


In [3]:
class cargo:
    def __init__(self,dim):
        self.dimensioni=dim

    def getW(self):
        return self.dimensioni[0]

    def getD(self):
        return self.dimensioni[1]



In [4]:
def plotSolution(solution, camion, packages):
    boxes = []

    width = [p.dimensioni[0] for p in packages]
    depth = [p.dimensioni[1] for p in packages]
    
    #bin
    container = Rectangle((0,0), camion.dimensioni[1], camion.dimensioni[0], facecolor='white')
    boxes.append(container)

    #items
    for a in solution:
        w = width[a[2]] if a[3] is 0 else depth[a[2]]
        d = depth[a[2]] if a[3] is 0 else width[a[2]]
        box = Rectangle((a[1],a[0]), d, w, facecolor='r')
        boxes.append(box)

    #plot
    facecolors = [b.get_facecolor() for b in boxes]

    collection = PatchCollection(boxes, facecolor=facecolors, alpha=0.4, edgecolor='black', linewidths=2)

    ax = plt.gca(xlabel='depth',ylabel='width')
    ax.add_collection(collection)
    ax.autoscale_view()
    ax.axis('equal')
    plt.draw_if_interactive()

In [5]:
def model(packages,cargo):
    model = Model("Model2D")  
    # importo dimensioni dei pacchi e del camion
    n=len(packages)

    w =[packages[i].getW() for i in range(n)]
    d =[packages[i].getD() for i in range(n)]

    print(w)
    print(d)

    W =cargo.getW()
    D =model.addVar(vtype="INTEGER",name="D")

    Md=sum(d)
    Mw=W+min(w)

    # definisco le variabili
    l =[[model.addVar(vtype="B",name ="l%d%d" % (i,j)) for i in range(n)] for j in range(n)]
    b =[[model.addVar(vtype="B",name ="b%d%d" % (i,j)) for i in range(n)] for j in range(n)]

    x =[model.addVar(vtype="INTEGER",name="x%d" % i) for i in range(n)]
    y =[model.addVar(vtype="INTEGER",name="y%d" % i) for i in range(n)]

    # definisco i constraints
    for i in range(n):
        for j in range(n):
            if(i < j):
                model.addCons(l[i][j] + l[j][i] + b[i][j] + b[j][i] >= 1)      #(1)
            if(i != j):
                model.addCons(x[i] - x[j] + Mw * l[i][j] <= Mw - w[i])         #(2)
                model.addCons(y[i] - y[j] + Md * b[i][j] <= Md - d[i])         #(3)

        model.addCons(x[i] <= W - w[i])                                        #(4)
        model.addCons(y[i] + d[i]<= D)                                         #(5)

    #funzione obiettivo
    model.setObjective(D, "minimize")
    model.data=x,y,l,b,w,d,W,D
    return model


In [6]:
def main():
    packages =[]
    for i in range(0,5):
        w=120
        d=80
        packages.append(box([w,d]))
    camion =cargo([250,1350])
    t=time.time()
    models =model(packages,camion)
    models.optimize()
    x,y,l,b,w,d,W,D= models.data
    s = []
    for i in range(len(packages)):
        s.append([models.getVal(x[i]), models.getVal(y[i]), i, 0])
    plotSolution(s, camion, packages)
    print s

In [ ]:
main()